In [ ]:
import os
import glob
import numpy as np
import pandas as pd

dir_path = './Korpus'
df_file = pd.DataFrame(columns=['file', 'content'])

for file_path in glob.glob(os.path.join(dir_path, '*.txt')):
    with open(file_path, 'r', encoding='utf-8') as file:
        file_name = os.path.basename(file_path)
        df_file.loc[len(df_file)] = [file_name, file.read()]



In [5]:
gold_standard = {
    'sår diabetes': {
        'Hælsår.txt',
        'Neuropatiske og neuroiskæmiske sår.txt',
        'Charcotfod.txt',
        'Diabetisk neuropati.txt',
        'Behandling for diabetes 2.txt',
        'Netdoktor.txt'
    },
    'livsstilssygdom': {
        'Livsstil.txt',
        'Risiko.txt',
        'Behandling for diabetes 2.txt',
        'Motion.txt',
        'Mad.txt',
        'Diabetes hos børn.txt'
    },
    'type 2-diabetes': {
        'Behandling for diabetes 2.txt',
        'Risiko.txt',
        'Motion.txt',
        'Mad.txt',
        'Livsstil.txt',
        'Diabetisk neuropati.txt'
    },
    'kost sukkersyge': {
        'Mad.txt',
        'Behandling for diabetes 2.txt',
        'Livsstil.txt',
        'Risiko.txt',
        'Motion.txt',
        'Diabetes hos børn.txt'
    }
}

In [ ]:
def SearchRoutine (input):
    files = []
    for idx, row in df_file.iterrows():
        if input.lower() in row['content'].lower():
            files.append(row['file'])
    return files
        
baseline_results = {}

for query in gold_standard:
    baseline_results[query] = SearchRoutine(query)


In [9]:
from sklearn.metrics import precision_score, recall_score, f1_score

def EvalQuary(gold_set, retrieved_list):
    retrieved_clean = [doc for doc in retrieved_list if doc is not None]

    # Union of all docs involved
    comparison_set = list(set(gold_set) | set(retrieved_clean))

    gold_vector = [1 if doc in gold_set else 0 for doc in comparison_set]
    retrieved_vector = [1 if doc in retrieved_clean else 0 for doc in comparison_set]

    precision = precision_score(gold_vector, retrieved_vector, zero_division=0)
    recall = recall_score(gold_vector, retrieved_vector, zero_division=0)
    f1 = f1_score(gold_vector, retrieved_vector, zero_division=0)

    return precision, recall, f1

scores = []

for query in gold_standard:
    gold = gold_standard[query]
    retrieved = baseline_results.get(query, [])

    precision, recall, f1 = EvalQuary(gold, retrieved)
    scores.append((query, precision, recall, f1))

    print(f"{query}: Precision={precision:.2f}, Recall={recall:.2f}, F1={f1:.2f}")

sår diabetes: Precision=0.00, Recall=0.00, F1=0.00
livsstilssygdom: Precision=1.00, Recall=0.17, F1=0.29
type 2-diabetes: Precision=0.50, Recall=0.17, F1=0.25
kost sukkersyge: Precision=0.00, Recall=0.00, F1=0.00


In [10]:
avg_precision = np.mean([s[1] for s in scores])
avg_recall = np.mean([s[2] for s in scores])
avg_f1 = np.mean([s[3] for s in scores])

print("\nAverage Scores (Top 6 only):")
print(f"Precision: {avg_precision:.2f}")
print(f"Recall:    {avg_recall:.2f}")
print(f"F1 Score:  {avg_f1:.2f}")


Average Scores (Top 6 only):
Precision: 0.38
Recall:    0.08
F1 Score:  0.13
